In [26]:
from collections import Counter
import numpy as np


In [20]:
def entropy(y):
    hist=np.bincount(y)
    ps=hist/len(y)
    return -np.sum([p*np.log2(p) for p in ps if p>0])
    

In [21]:
test_y=[0,0,0,0,1]
print(entropy(test_y))

0.7219280948873623


In [31]:
class Node:
    
    def __init__(self,feature=None,threshold=None,left=None,right=None,*,value=None):
        self.feature=feature
        self.value=value
        self.threshold=threshold
        self.left=left
        self.right=right
        
    def isleaf(self):
        return self.value is not None

In [44]:
class DecisionTree:
    
    def __init__(self,min_samples_split=2,max_depth=10,n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None
        
    def find_most_common(self,y):
        counter=Counter(y)
        most_common=counter.most_common(1)[0][0]
        return most_common
        
    def grow_tree(self,x,y,depth=0):
        
        n_samples,n_feats=x.shape
        n_labels=len(np.unique(y))
        
        #stopping criteria
        if (depth>=self.max_depth or n_samples<self.min_samples_split or n_labels==1):
            leaf_value=self.find_most_common(y)
            return Node(value=leaf_value)
        
        feature_idx=np.random.choice(n_feats,self.n_features,replace=False)
        # Find the best split
        
        best_feature,best_threshold=self.best_split(x,y,feature_idx)
        
        #find children
        left_idx,right_idx=self.split(x[:,best_feature],best_threshold)
        
        left=self.grow_tree(x[left_idx,:],y[left_idx],depth+1)
        right=self.grow_tree(x[right_idx,:],y[right_idx],depth+1)
        return Node(best_feature,best_threshold,left,right)
    
    
    def best_split(self,x,y,feature_idx):
        best_gain=-1
        split_idx,split_threshold=None,None
        
        for f in feature_idx:
            x_colm=x[:,f]
            threshold_values=np.unique(x_colm)
            
            for t in threshold_values:
                # Calculate the information gain
                gain=self.info_gain(y,x_colm,t)
                
                if gain>best_gain:
                    best_gain=gain
                    split_idx=f
                    split_threshold=t
        return split_idx,split_threshold
    
    
    def info_gain(self,y,column,threshold):
       
        #parent entropy
        parent_entropy=self.entropy(y)
        
        #create children
        left_idx,right_idx=self.split(column,threshold)
        
        if len(left_idx)==0 or len(right_idx)==0:
            return 0
        
        #calculate weighted average entropy of children
        n=len(y)
        n_l,n_r=len(left_idx),len(right_idx)
        
        e_l,e_r=self.entropy(y[left_idx]),self.entropy(y[right_idx])
        
        entropy_child=n_l/n*e_l+n_r/n*e_r
        
        information_gain=parent_entropy-entropy_child
        
        return information_gain
        
        
    
    def split(self,x_column,thres):
        left_idx=np.argwhere(x_column<=thres).flatten()
        right_idx=np.argwhere(x_column>thres).flatten()
        return left_idx,right_idx
        
        
        # calculate information gain
    def entropy(self,y):
        hist=np.bincount(y)
        ps=hist/len(y)
        return -np.sum([p*np.log2(p) for p in ps if p>0])
    
    
    def fit(self,x,y):
        # growing our tree
        
        self.n_features=x.shape[1] if self.n_features is None else min(self.n_features,x.shape[1])
        self.root=self.grow_tree(x,y)
        
    
    
    def predict(self,test):
        # traversing our tree
        return np.array([self.traverse(x,self.root) for x in test])
    
    
    def traverse(self,x,node):
        if node.isleaf():
            return node.value

        if x[node.feature] <= node.threshold:
            return self.traverse(x, node.left)
        return self.traverse(x, node.right)
        

In [45]:
if __name__ == "__main__":
    # Imports
    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    def accuracy(y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

    data = datasets.load_breast_cancer()
    x, y = data.data, data.target

    xtrain, xtest, ytrain, ytest = train_test_split(
        x, y, test_size=0.2, random_state=1234
    )
    
    clf=DecisionTree(max_depth=10)
    clf.fit(xtrain,ytrain)
    ypred=clf.predict(xtest)
    
    acc_percent=accuracy(ytest,ypred)
    
    print("{:.2f}".format(acc_percent))

0.93
